In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"  # Show all outputs in Jupyter

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
RAG系统测试脚本

这个脚本用于测试RAG系统的各个组件和整体功能。
"""

import os
import sys
import time
import json
from pathlib import Path

# 添加项目根目录到系统路径
project_root = Path('../')  # Since this variable is already defined with value WindowsPath('.') in another cell
sys.path.insert(0, str(project_root))

'\nRAG系统测试脚本\n\n这个脚本用于测试RAG系统的各个组件和整体功能。\n'

In [3]:
# 导入自定义模块
#sys.path.append("..")

from src.document_loader import load_documents
from src.text_chunker import split_documents, ChunkerFactory
from src.embeddings import generate_embeddings, EmbedderFactory
from src.vector_store import SimpleVectorStore, FAISSVectorStore, ChromaVectorStore
from src.retriever import RetrieverFactory
from src.generator import GeneratorFactory
from src.rag_pipeline import RAGPipeline, create_rag_pipeline

警告: FAISS未安装，将无法使用FAISSVectorStore
警告: ChromaDB未安装，将无法使用ChromaVectorStore


In [6]:
def test_document_loading():
    """测试文档加载功能"""
    print("\n===== 测试文档加载 =====")
    data_dir = os.path.join(project_root, "data")
    os.listdir(data_dir)
    documents = load_documents(data_dir)
    
    print(f"加载了 {len(documents)} 个文档")
    for i, doc in enumerate(documents):
        print(f"文档 {i+1}:")
        print(f"  - 文件名: {doc['metadata']['source']}")
        print(f"  - 内容长度: {len(doc['content'])} 字符")
        print(f"  - 内容预览: {doc['content'][:100]}...")
    
    return documents

In [7]:
documents = test_document_loading()
documents[0]


===== 测试文档加载 =====


加载文档: 100%|██████████| 2/2 [00:00<?, ?it/s]

加载了 2 个文档
文档 1:
  - 文件名: ..\data\test_document1.txt
  - 内容长度: 629 字符
  - 内容预览: 人工智能简介 人工智能Artificial Intelligence简称AI是计算机科学的一个分支它致力于研究和开发能够模拟延伸和扩展人类智能的理论方法技术及应用系统  人工智能的主要领域  机器学习...
文档 2:
  - 文件名: ..\data\test_document2.txt
  - 内容长度: 823 字符
  - 内容预览: 大语言模型简介 大语言模型Large Language Models简称LLMs是一种基于深度学习的自然语言处理模型它们通过在海量文本数据上训练能够理解和生成人类语言  大语言模型的发展历程 大语言模...


{'content': '人工智能简介 人工智能Artificial Intelligence简称AI是计算机科学的一个分支它致力于研究和开发能够模拟延伸和扩展人类智能的理论方法技术及应用系统  人工智能的主要领域  机器学习 机器学习是人工智能的核心它使计算机能够从数据中学习而无需明确编程主要方法包括监督学习无监督学习和强化学习  深度学习 深度学习是机器学习的一个子集它使用多层神经网络来模拟人脑的工作方式深度学习在图像识别语音识别和自然语言处理等领域取得了突破性进展  自然语言处理 自然语言处理NLP使计算机能够理解解释和生成人类语言它是语音助手机器翻译和情感分析等应用的基础  计算机视觉 计算机视觉使机器能够从图像或视频中获取信息理解视觉世界它广泛应用于人脸识别自动驾驶和医学影像分析等领域  人工智能的应用 1. 医疗保健疾病诊断药物发现个性化治疗 2. 金融风险评估欺诈检测算法交易 3. 交通自动驾驶交通流量优化路线规划 4. 教育个性化学习自动评分智能辅导 5. 制造业预测性维护质量控制供应链优化  人工智能的挑战与伦理问题 尽管人工智能带来了巨大的机遇但也面临着诸多挑战和伦理问题如隐私保护算法偏见就业影响和安全风险等解决这些问题需要技术专家政策制定者和社会各界的共同努力  人工智能的未来 随着技术的不断进步人工智能将继续改变我们的生活和工作方式通用人工智能AGI和超级人工智能ASI的发展可能会带来更加深远的影响人类需要谨慎而明智地引导这一技术的发展方向',
 'metadata': {'source': '..\\data\\test_document1.txt',
  'filename': 'test_document1.txt',
  'filetype': 'txt'}}

In [42]:
def test_text_chunking(documents):
    """测试文本分块功能"""
    print("\n===== 测试文本分块 =====")
    
    # 测试不同的分块方法
    chunker_types = ["paragraph", "fixed_size"]
    chunk_sizes = [500, 1000]
    
    results = {}
    
    for chunker_type in chunker_types:
        for chunk_size in chunk_sizes:
            print(f"\n使用 {chunker_type} 分块器，块大小 {chunk_size}:")
            
            # 分块
            start_time = time.time()
            chunks = split_documents(
                documents,
                chunker_type=chunker_type,
                chunk_size=chunk_size,
                chunk_overlap=100
            )
            end_time = time.time()
            
            print(f"  - 生成了 {len(chunks)} 个文本块")
            print(f"  - 处理时间: {end_time - start_time:.4f} 秒")
            
            # 显示一些块的信息
            if chunks:
                print(f"  - 第一个块大小: {len(chunks[0]['content'])} 字符")
                print(f"  - 第一个块内容预览: {chunks[0]['content'][:100]}...")
            
            results[(chunker_type, chunk_size)] = chunks
    
    # 返回段落分块器、块大小1000的结果用于后续测试
    return results.get(("paragraph", 1000), [])

In [43]:
def test_embeddings(chunks):
    """测试向量嵌入功能"""
    print("\n===== 测试向量嵌入 =====")
    
    # 使用sentence_transformer嵌入器
    embedder_type = "sentence_transformer"
    model_name = "paraphrase-multilingual-MiniLM-L12-v2"
    
    print(f"使用 {embedder_type} 嵌入器，模型 {model_name}:")
    
    # 创建嵌入器
    embedder = EmbedderFactory.get_embedder(
        embedder_type=embedder_type,
        model_name=model_name
    )
    
    # 生成嵌入向量
    start_time = time.time()
    chunks_with_embeddings = embedder.generate_embeddings(chunks)
    end_time = time.time()
    
    print(f"  - 为 {len(chunks_with_embeddings)} 个文本块生成了嵌入向量")
    print(f"  - 处理时间: {end_time - start_time:.4f} 秒")
    
    # 显示嵌入向量的维度
    if chunks_with_embeddings:
        embedding_dim = len(chunks_with_embeddings[0]['embedding'])
        print(f"  - 嵌入向量维度: {embedding_dim}")
    
    return chunks_with_embeddings


def test_vector_store(chunks_with_embeddings):
    """测试向量存储功能"""
    print("\n===== 测试向量存储 =====")
    
    # 测试SimpleVectorStore
    print("\n测试 SimpleVectorStore:")
    simple_store = SimpleVectorStore()
    
    # 添加文档
    start_time = time.time()
    simple_store.add_documents(chunks_with_embeddings)
    end_time = time.time()
    
    print(f"  - 添加了 {len(chunks_with_embeddings)} 个文档到向量存储")
    print(f"  - 处理时间: {end_time - start_time:.4f} 秒")
    
    # 测试搜索
    if chunks_with_embeddings:
        query_vector = chunks_with_embeddings[0]['embedding']  # 使用第一个文档的向量作为查询
        results = simple_store.search(query_vector, top_k=3)
        
        print(f"  - 搜索结果数量: {len(results)}")
        for i, result in enumerate(results):
            print(f"    结果 {i+1}: 相似度分数 = {result['score']:.4f}")
            print(f"    内容预览: {result['content'][:100]}...")
    
    # 测试保存和加载
    save_dir = os.path.join(project_root, "examples", "test_vector_store")
    os.makedirs(save_dir, exist_ok=True)
    
    # 保存
    simple_store.save(save_dir)
    print(f"  - 向量存储已保存到 {save_dir}")
    
    # 加载
    new_store = SimpleVectorStore()
    new_store.load(save_dir)
    print(f"  - 从 {save_dir} 加载了向量存储")
    print(f"  - 加载的文档数量: {len(new_store.documents)}")
    
    # 如果有FAISS可用，也测试FAISSVectorStore
    try:
        import faiss
        print("\n测试 FAISSVectorStore:")
        faiss_store = FAISSVectorStore()
        
        # 添加文档
        start_time = time.time()
        faiss_store.add_documents(chunks_with_embeddings)
        end_time = time.time()
        
        print(f"  - 添加了 {len(chunks_with_embeddings)} 个文档到FAISS向量存储")
        print(f"  - 处理时间: {end_time - start_time:.4f} 秒")
        
        # 测试搜索
        if chunks_with_embeddings:
            query_vector = chunks_with_embeddings[0]['embedding']
            results = faiss_store.search(query_vector, top_k=3)
            
            print(f"  - 搜索结果数量: {len(results)}")
            for i, result in enumerate(results):
                print(f"    结果 {i+1}: 相似度分数 = {result['score']:.4f}")
    except ImportError:
        print("FAISS未安装，跳过FAISSVectorStore测试")
    
    return simple_store


def test_retriever(vector_store, chunks_with_embeddings):
    """测试检索器功能"""
    print("\n===== 测试检索器 =====")
    
    # 创建检索器
    retriever = RetrieverFactory.get_retriever(
        retriever_type="vector",
        vector_store=vector_store,
        embedder_type="sentence_transformer",
        model_name="paraphrase-multilingual-MiniLM-L12-v2"
    )
    
    # 测试检索
    test_queries = [
        "什么是人工智能？",
        "大语言模型有哪些局限性？",
        "RAG系统如何工作？"
    ]
    
    for query in test_queries:
        print(f"\n查询: '{query}'")
        
        start_time = time.time()
        results = retriever.retrieve(query, top_k=3)
        end_time = time.time()
        
        print(f"  - 检索时间: {end_time - start_time:.4f} 秒")
        print(f"  - 检索结果数量: {len(results)}")
        
        for i, result in enumerate(results):
            print(f"    结果 {i+1}: 相似度分数 = {result['score']:.4f}")
            print(f"    内容预览: {result['content'][:100]}...")
    
    return retriever


def test_generator(retriever):
    """测试生成器功能"""
    print("\n===== 测试生成器 =====")
    
    # 创建模板生成器
    generator = GeneratorFactory.get_generator(generator_type="template")
    
    # 测试生成
    test_queries = [
        "什么是人工智能？",
        "大语言模型有哪些局限性？",
        "RAG系统如何工作？"
    ]
    
    for query in test_queries:
        print(f"\n查询: '{query}'")
        
        # 检索相关文档
        retrieved_docs = retriever.retrieve(query, top_k=3)
        
        # 生成回答
        start_time = time.time()
        answer = generator.generate(query, retrieved_docs)
        end_time = time.time()
        
        print(f"  - 生成时间: {end_time - start_time:.4f} 秒")
        print(f"  - 生成的回答:\n{answer}")


def test_rag_pipeline():
    """测试完整的RAG流程"""
    print("\n===== 测试完整RAG流程 =====")
    
    # 创建RAG流程
    config_path = os.path.join(project_root, "config.json")
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    rag = RAGPipeline(config)
    
    # 索引文档
    data_dir = os.path.join(project_root, "data")
    print(f"\n索引文档目录: {data_dir}")
    
    start_time = time.time()
    rag.index_documents(data_dir)
    end_time = time.time()
    
    print(f"  - 索引时间: {end_time - start_time:.4f} 秒")
    
    # 测试查询
    test_queries = [
        "什么是人工智能？",
        "大语言模型有哪些局限性？",
        "RAG系统如何工作？"
    ]
    
    for query in test_queries:
        print(f"\n查询: '{query}'")
        
        start_time = time.time()
        answer = rag.query(query)
        end_time = time.time()
        
        print(f"  - 查询时间: {end_time - start_time:.4f} 秒")
        print(f"  - 生成的回答:\n{answer}")
    
    # 测试保存和加载
    save_dir = os.path.join(project_root, "examples", "test_rag_pipeline")
    os.makedirs(save_dir, exist_ok=True)
    
    # 保存
    rag.save(save_dir)
    print(f"\nRAG流程已保存到 {save_dir}")
    
    # 加载
    new_rag = create_rag_pipeline()
    new_rag.load(save_dir)
    print(f"从 {save_dir} 加载了RAG流程")
    
    # 测试加载后的查询
    query = "RAG系统的优势是什么？"
    print(f"\n加载后查询: '{query}'")
    
    answer = new_rag.query(query)
    print(f"  - 生成的回答:\n{answer}")




In [ ]:
def main():
    """主函数"""
    print("开始测试RAG系统...\n")
    
    # 测试各个组件
    documents = test_document_loading()
    chunks = test_text_chunking(documents[0])
    chunks_with_embeddings = test_embeddings(chunks)
    vector_store = test_vector_store(chunks_with_embeddings)
    retriever = test_retriever(vector_store, chunks_with_embeddings)
    test_generator(retriever)
    
    # 测试完整流程
    test_rag_pipeline()


if __name__ == "__main__":
    main()

开始测试RAG系统...


===== 测试文档加载 =====


加载文档: 100%|██████████| 2/2 [00:00<?, ?it/s]

加载了 2 个文档
文档 1:
  - 文件名: ..\data\test_document1.txt
  - 内容长度: 629 字符
  - 内容预览: 人工智能简介 人工智能Artificial Intelligence简称AI是计算机科学的一个分支它致力于研究和开发能够模拟延伸和扩展人类智能的理论方法技术及应用系统  人工智能的主要领域  机器学习...
文档 2:
  - 文件名: ..\data\test_document2.txt
  - 内容长度: 823 字符
  - 内容预览: 大语言模型简介 大语言模型Large Language Models简称LLMs是一种基于深度学习的自然语言处理模型它们通过在海量文本数据上训练能够理解和生成人类语言  大语言模型的发展历程 大语言模...

===== 测试文本分块 =====

使用 paragraph 分块器，块大小 500:
